<a href="https://colab.research.google.com/github/ynaowusu/protein-folding-quantum-algorithms/blob/main/hamiltonian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install qiskit #this is just me intalling qiskit into our notebook
!pip install matplotlib plotly #since it says we need a 3d structure to simulate the lattice and any other 3d elements
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.5 MB/s eta 0:00:00


In [11]:
import numpy as np
import pytest
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np
from typing import List, Dict, Tuple


In [12]:
class ProtienFoldVector: #added a class here because the vector encdoings can be seen as an object
  def __init_(self):
    self.lattice_vector = {
        "A":{
            0:[(1,1,1)], #|00> This is for the first round of qubits reprsents x,y,z
            1:[(1,-1,-1)], #|10>
            2:[(-1,1,-1)], #|01>
            3:[(-1,-1,-1)] #|11>
        },
        "B":{
            0:[(1,1,-1)], #|00>
            1:[(1,-1,1)], #|10>
            2:[(-1,1,1)], #|01>
            3:[(-1,-1,-1)] #11>
        }
    }


        #proteins fold in 3D space and here we're working with a simplified lattice model w tetrahedral directions. There are two kinds of alternating sites in the protein chain: 'A' and 'B'.
        #every turn corresponds to a direction in 3D space either x,y or z
        #The vectors  [1, 1, 1]) are unit steps in different directions based on the protein structure model.







    pass


    #vectors go here



In [13]:
"""
  - Sparse (4-qubit per turn) one-hot encoding on the alternating A/B diamond lattice.
  - Qiskit helpers to allocate and prepare the configuration-qubit register.
  - Decoding measured bitstrings into 3D backbone turn vectors.

Usage:
  from hamiltonian import (
      create_configuration_circuit,
      decode_measurement,
      SparseDiamondEncoder
  )

  qc, cfg, meas = create_configuration_circuit(N)
  # ... add variational ansatz, measure, run on backend ...
  counts = backend.run(qc).result().get_counts()
  turns = decode_measurement(counts, N)
"""


class SparseDiamondEncoder:
    """Encodes/decodes backbone turns on a diamond (tetrahedral) lattice."""

    # sublattice A basis (even sites)
    _A: Dict[int,np.ndarray] = {
        0: np.array([+1,+1,+1]),
        1: np.array([+1,-1,-1]),
        2: np.array([-1,+1,-1]),
        3: np.array([-1,-1,+1]),
    }
    # sublattice B basis (odd sites)
    _B: Dict[int,np.ndarray] = {
        0: np.array([+1,+1,-1]),
        1: np.array([+1,-1,+1]),
        2: np.array([-1,+1,+1]),
        3: np.array([-1,-1,-1]),
    }

    @staticmethod
    def site_parity(i) :
        """Return 'A' if turn index i is even (sublattice A), else 'B'."""
        return 'A' if i % 2 == 0 else 'B'

    @classmethod
    def block_to_vec(cls, block, i) :
        """
        Decode one-hot 4-bit string (e.g. '1000') at turn index i into a 3D step vector.
        Raises ValueError if block is not one-hot of length 4.
        """
        if len(block) != 4 or block.count('1') != 1:
            raise ValueError(f"Invalid block {block}, must be one-hot length 4.")
        axis = ['1000','0100','0010','0001'].index(block)
        if cls.site_parity(i) == 'A':
            table = cls._A
        else:
            table = cls._B
        return table[axis]
    @classmethod
    def bitstring_to_turns(cls, bitstr, N) :
        """
        Convert full measured bitstring (length=4*(N-3)) into list of N-1 turn vectors:
        - t1 = (+1,+1,+1)
        - t2 = (+1,-1,-1)
        - t3..t_{N-1} from blocks of 4 bits
        """
        expected = 4*(N-3)
        if len(bitstr) != expected:
            raise ValueError(f"Expected bitstring length {expected}, got {len(bitstr)}")
        turns: List[np.ndarray] = []
        # fixed first two turns
        turns.append(cls._A[0])  # t1
        turns.append(cls._A[1])  # t2
        # decode remaining
        for k, start in enumerate(range(0, expected, 4), start=3):
            block = bitstr[start:start+4]
            turns.append(cls.block_to_vec(block, k))
        return turns


def create_configuration_circuit(N):
    """
    Allocate 4*(N-3) config qubits in a register, apply Hadamard to each,
    and attach classical bits for measurement.
    Returns (qc, cfg, meas).
    """
    n_cfg = 4*(N-3)
    cfg = QuantumRegister(n_cfg, 'cfg')
    meas = ClassicalRegister(n_cfg, 'meas')
    qc = QuantumCircuit(cfg, meas)
    qc.h(cfg)
    return qc, cfg, meas


def decode_measurement(counts, N):
    """
    From measurement counts, pick the most frequent bitstring and decode into turn vectors.
    """
    bitstr = max(counts, key=counts.get)
    return SparseDiamondEncoder.bitstring_to_turns(bitstr, N)

In [27]:

"""
Hamiltonian  defines the cost Hamiltonian terms for a protein fold on a diamond lattice:

1. Growth Constraint (H_gc): penalises consecutive backtracking bonds.
2. Overlap Penalty (H_ov): penalises any pair of beads occupying the same site.
3. Contact Reward (H_ct): rewards non-bonded nearest neighbours.
4. Chirality Constraint (H_ch): enforces correct handedness per sublattice.
5. Interaction Stub (H_in): placeholder for shell-based interaction qubits.

Each method returns a classical float energy for the decoded turn list.
"""

def T(t_i: np.ndarray, t_j: np.ndarray) -> int:
    """
    Indicator: 1 if t_i and t_j lie on same lattice axis or opposite, else 0.
    """
    return int(np.array_equal(t_i, t_j) or np.array_equal(t_i, -t_j))


class Hamiltonian:
    def __init__(
        self,
        turns: List[np.ndarray],
        back_penalty_weight: float = 30.0,
        overlap_weight: float = 50.0,
        contact_reward: float = 10.0,
        chirality_weight: float = 40.0,
        interaction_weights=None,
    ):
        """
        Parameters:
        -----------
        turns : List of N-1 3D step vectors (np.ndarray).
        back_penalty_weight : λ_back for growth constraint.
        overlap_weight : λ_ov for overlap penalty.
        contact_reward : ε for contact reward (>0).
        chirality_weight : λ_chir for chirality penalty.
        """
        self.turns = turns
        self.N = len(turns) + 1
        self.λ_back = back_penalty_weight
        self.λ_ov = overlap_weight
        self.ε = contact_reward
        self.λ_chir = chirality_weight
        self.interaction_weights = interaction_weights or {}
    def growth_constraint(self):
        """
        H_gc = λ_back * Σ_{i=3..N-1} T(t_i, t_{i+1}).
        """
        H_gc = 0.0
        for i in range(2, self.N - 2):
            H_gc += self.λ_back * T(self.turns[i], self.turns[i + 1])
        return H_gc

    def overlap_penalty(self):
        """
        H_ov = λ_ov * Σ_{i<j} overlap(i,j),
        where overlap(i,j)=1 if positions coincide, else 0.
        """
        pos = [(0, 0, 0)]
        for t in self.turns:
            prev = np.array(pos[-1])
            pos.append(tuple(prev + t))
        H_ov = 0.0
        #threshold for nearest neighbour on diamond lattice
        for i in range(self.N):
            for j in range(i + 1, self.N):
                if pos[i] == pos[j]:
                    H_ov += self.λ_ov
        return H_ov

    def contact_reward(self):
        """
        H_ct = -ε * Σ_{|i-j|>2 and nearest neighbours} 1,
        where nearest neighbours if Euclidean distance == 1.
        """
        pos = [(0, 0, 0)]
        for t in self.turns:
            prev = np.array(pos[-1])
            pos.append(tuple(prev + t))
        H_ct = 0.0
        # threshold for nearest neighbour on diamond lattice
        nn_dist = np.sqrt(3)
        tol = 1e-6
        for i in range(self.N):
            # |i-j|>2
            for j in range(i + 3, self.N):
                d = np.linalg.norm(np.array(pos[i]) - np.array(pos[j]))
                if abs(d - nn_dist) < tol:
                    H_ct += -self.ε
        return H_ct
    def chirality_constraint(self):
        """
        H_ch = λ_chir * Σ_{i=2..N-2} indicator of wrong handedness;
        uses scalar triple product and sublattice parity.
        """
        H_ch = 0.0
        for i in range(2, self.N - 2):
            v_prev = self.turns[i - 1]
            v_curr = self.turns[i]
            v_side = self.turns[i + 1]
            triple = np.dot(np.cross(v_prev, v_curr), v_side)
            parity = (1 - (-1) ** i) // 2
            # expect triple>0 on A (parity=0), <0 on B (parity=1)
            if not ((parity == 0 and triple > 0) or (parity == 1 and triple < 0)):
                H_ch += self.λ_chir
        return H_ch


    def interaction_constraint(self):

        H_in = 0.0
        # build bead positions
        pos = [(0, 0, 0)]
        for t in self.turns:
            prev = np.array(pos[-1])
            pos.append(tuple(prev + t))
        tol = 1e-6
        for shell, weight in self.interaction_weights.items():
            for i in range(self.N):
                for j in range(i + 3, self.N):
                    d = np.linalg.norm(np.array(pos[i]) - np.array(pos[j]))
                    if abs(d - shell) < tol:
                        H_in += weight
        return H_in

    def total_energy(self):
        """
        Total classical energy: H_gc + H_ov + H_ct + H_ch + H_in
        """
        return (
            self.growth_constraint()
            + self.overlap_penalty()
            + self.contact_reward()
            + self.chirality_constraint()
            + self.interaction_constraint()
        )

In [26]:
#tests
basis = [
    np.array([+1, +1, +1]),
    np.array([+1, -1, -1]),
    np.array([-1, +1, -1]),
    np.array([-1, -1, +1]),
]

def test_T_indicator():
    assert T(basis[0], basis[0]) == 1
    assert T(basis[0], -basis[0]) == 1
    assert T(basis[0], basis[1]) == 0
    print("all test_t_indicator tests passed")
test_T_indicator()


def test_growth_constraint_no_backtrack():
    # no consecutive same-axis turns => zero penalty
    turns = [basis[0], basis[1], basis[2], basis[3], basis[0]]
    H = Hamiltonian(turns, back_penalty_weight=5)
    assert H.growth_constraint() == 0
    print("all growth_constraint_no_backtrack tests passed")
test_growth_constraint_no_backtrack()
def test_overlap_penalty_detects_collision():
    # create a self-overlap by reversing a turn
    # turns: t1, t2 fixed, then t3 reverses direction of t2 => overlap after 3 steps
    turns = [basis[0], basis[1], -basis[1], basis[2]]
    H = Hamiltonian(turns, overlap_weight=7)
    ov = H.overlap_penalty()
    assert ov > 0
    assert ov % 7 == 0  # multiple of penalty weight
    print("all test_overlap_penalty_detects_collision tests passed")
test_overlap_penalty_detects_collision()
'''
  # create a nearest-neighbour non-bond contact: t3 brings bead next to bead 0
    # assemble positions manually for clarity
'''

def test_contact_reward_detects_neighbor():

    turns = [basis[0], basis[1], -basis[0], basis[2]]
    H = Hamiltonian(turns, contact_reward=3)
    cr = H.contact_reward()
    assert cr < 0
    assert cr % -3 == 0
    print("all test_contact_reward_detects_neighbor tests passed")
test_contact_reward_detects_neighbor()
def test_total_energy_combines_terms():
    turns = [basis[0], basis[1], basis[2], basis[3], basis[0]]
    H = Hamiltonian(
        turns,
        back_penalty_weight=5,
        overlap_weight=6,
        contact_reward=2,
        chirality_weight=4
    )
    total = H.total_energy()
    # For this random turn list no overlap or backtrack or contact should occur,
    # chirality may contribute but total should be non-negative
    assert total >= 0
    print("all total_energy_combines_terms tests passed")
test_total_energy_combines_terms()

all test_t_indicator tests passed
all growth_constraint_no_backtrack tests passed
all test_overlap_penalty_detects_collision tests passed
all test_contact_reward_detects_neighbor tests passed
all total_energy_combines_terms tests passed
